# Smart Waste Management System

A Python notebook that optimizes waste collection by:
1. Segmenting cities into districts using map coloring algorithms
2. Optimizing collection routes using backtracking algorithms

## Setup and Imports

In [ ]:
# Install required packages if needed
!pip install networkx matplotlib

In [ ]:
import random
import csv
import math
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image

## Model Classes

In [ ]:
class District:
    def __init__(self, district_id, name, adjacent_districts=None):
        self.district_id = district_id
        self.name = name
        self.adjacent_districts = adjacent_districts or []
        self.color = None
        self.waste_bins = []
    
    def add_adjacent_district(self, district):
        if district not in self.adjacent_districts:
            self.adjacent_districts.append(district)
            district.adjacent_districts.append(self)
    
    def add_waste_bin(self, waste_bin):
        self.waste_bins.append(waste_bin)


class WasteBin:
    def __init__(self, bin_id, location, fill_level, capacity=100, bin_type="Mixed", container_type="Standard"):
        self.bin_id = bin_id
        self.location = location  # (x, y) coordinates
        self.capacity = capacity  # maximum capacity in kg or percentage
        self.current_level = fill_level  # current fill level (FL_B in dataset)
        self.emptying_needed = self.current_level > 65  # Based on fill level threshold
        self.container_type = container_type  # From Container Type in dataset
        self.bin_type = bin_type  # From Recyclable fraction in dataset
    
    @property
    def fill_percentage(self):
        return (self.current_level / self.capacity) * 100


class Truck:
    def __init__(self, truck_id, capacity, bin_type_specialty=None):
        self.truck_id = truck_id
        self.capacity = capacity  # maximum capacity in kg
        self.current_load = 0
        self.route = []  # list of WasteBin objects to visit
        self.bin_type_specialty = bin_type_specialty  # Trucks can specialize in certain waste types
    
    def add_bin_to_route(self, waste_bin):
        if self.can_handle(waste_bin):
            self.route.append(waste_bin)
            return True
        return False
    
    def can_handle(self, waste_bin):
        # Check if truck has capacity and handles this bin type (if specialization exists)
        if self.bin_type_specialty and waste_bin.bin_type != self.bin_type_specialty and waste_bin.bin_type != "Mixed":
            return False
        return self.current_load + waste_bin.current_level <= self.capacity
    
    def collect_waste(self, waste_bin):
        if self.can_handle(waste_bin):
            self.current_load += waste_bin.current_level
            waste_bin.current_level = 0
            return True
        return False

## Map Coloring Algorithm

In [ ]:
def create_district_graph(districts):
    """Create a graph representing districts and their adjacencies"""
    G = nx.Graph()
    
    # Add nodes
    for district in districts:
        G.add_node(district.district_id, district=district)
    
    # Add edges for adjacent districts
    for district in districts:
        for adj_district in district.adjacent_districts:
            G.add_edge(district.district_id, adj_district.district_id)
    
    return G

def map_coloring(districts, colors=None):
    """
    Assign colors to districts such that no adjacent districts have the same color
    using a greedy algorithm.
    """
    if colors is None:
        colors = ["red", "green", "blue", "yellow"]
    
    # Sort districts by degree (number of adjacent districts)
    graph = create_district_graph(districts)
    sorted_districts = sorted(
        districts, 
        key=lambda d: len(d.adjacent_districts), 
        reverse=True
    )
    
    for district in sorted_districts:
        # Get colors of adjacent districts
        used_colors = {
            adj_district.color for adj_district in district.adjacent_districts
            if adj_district.color is not None
        }
        
        # Find first available color
        for color in colors:
            if color not in used_colors:
                district.color = color
                break
    
    return districts

def visualize_districts(districts):
    """Visualize the district map with colors"""
    G = create_district_graph(districts)
    
    # Get position for nodes - this is simplified and would need real coordinates in a real system
    pos = nx.spring_layout(G)
    
    # Get colors for nodes
    colors = [districts[i-1].color for i in G.nodes]
    
    plt.figure(figsize=(10, 8))
    nx.draw(
        G, 
        pos, 
        node_color=colors,
        with_labels=True, 
        node_size=1500, 
        font_size=10,
        font_weight='bold'
    )
    plt.title("District Map Coloring")
    plt.show()

## Route Optimization

In [ ]:
def distance(bin1, bin2):
    """Calculate Euclidean distance between two waste bins"""
    x1, y1 = bin1.location
    x2, y2 = bin2.location
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def total_route_distance(route):
    """Calculate the total distance of a route"""
    total = 0
    for i in range(len(route) - 1):
        total += distance(route[i], route[i + 1])
    return total

def optimize_route_backtracking(truck, waste_bins, start_point=None):
    """
    Use backtracking to find optimal route for a truck to collect waste.
    Returns the optimized route and its distance.
    
    Args:
        truck: The truck object
        waste_bins: List of waste bins that need collection
        start_point: Starting location coordinates (x,y)
    """
    if not waste_bins:
        return [], 0
    
    # Filter bins that need emptying and those matching truck specialty
    priority_bins = [bin for bin in waste_bins if bin.emptying_needed]
    
    # If truck has specialty, prioritize bins of that type
    if truck.bin_type_specialty:
        matching_bins = [bin for bin in priority_bins 
                        if bin.bin_type == truck.bin_type_specialty or bin.bin_type == "Mixed"]
        if matching_bins:
            priority_bins = matching_bins
    
    # If no priority bins, use regular waste bins
    if not priority_bins:
        priority_bins = waste_bins
    
    # Sort bins by fill level to prioritize fuller bins
    priority_bins = sorted(priority_bins, key=lambda x: x.current_level, reverse=True)
    
    best_route = []
    best_distance = float('inf')
    visited = [False] * len(priority_bins)
    current_route = []
    current_load = 0
    
    def backtrack(position, current_distance):
        nonlocal best_route, best_distance, current_route, current_load
        
        # If all bins that can fit are visited or truck is full
        if all(visited) or current_load >= truck.capacity * 0.9:
            # Complete path found, check if it's better
            if current_distance < best_distance:
                best_distance = current_distance
                best_route = current_route.copy()
            return
        
        for i in range(len(priority_bins)):
            # Skip if already visited or bin would overload truck
            if visited[i] or current_load + priority_bins[i].current_level > truck.capacity:
                continue
                
            # Skip if truck specializes in a waste type and this bin doesn't match (unless mixed)
            if (truck.bin_type_specialty and 
                priority_bins[i].bin_type != truck.bin_type_specialty and 
                priority_bins[i].bin_type != "Mixed"):
                continue
                
            # Calculate distance from current position to this bin
            dist_to_bin = 0
            if position is not None:
                if current_route:
                    dist_to_bin = distance(current_route[-1], priority_bins[i])
                else:
                    # Distance from start point to first bin
                    x, y = position
                    start_bin = WasteBin(-1, position, 0, 0)  # Dummy bin for calculation
                    dist_to_bin = distance(start_bin, priority_bins[i])
            
            # Try this bin
            visited[i] = True
            current_route.append(priority_bins[i])
            current_load += priority_bins[i].current_level
            
            # Recursive call
            backtrack(priority_bins[i].location, current_distance + dist_to_bin)
            
            # Backtrack
            visited[i] = False
            current_route.pop()
            current_load -= priority_bins[i].current_level
    
    # Start the backtracking process
    backtrack(start_point, 0)
    return best_route, best_distance

def assign_trucks_to_districts(trucks, districts):
    """Assign trucks to districts based on waste volume and type"""
    assignments = {}
    
    # Sort districts by total waste volume
    districts_by_volume = sorted(
        districts,
        key=lambda d: sum(bin.current_level for bin in d.waste_bins if bin.emptying_needed),
        reverse=True
    )
    
    # Calculate waste type composition per district for better truck assignment
    district_stats = {}
    for district in districts:
        recyclable = sum(bin.current_level for bin in district.waste_bins 
                         if bin.bin_type == "Recyclable" and bin.emptying_needed)
        non_recyclable = sum(bin.current_level for bin in district.waste_bins 
                            if bin.bin_type == "Non Recyclable" and bin.emptying_needed)
        mixed = sum(bin.current_level for bin in district.waste_bins 
                   if bin.bin_type == "Mixed" and bin.emptying_needed)
        total = recyclable + non_recyclable + mixed
        
        district_stats[district.district_id] = {
            "recyclable": recyclable,
            "non_recyclable": non_recyclable,
            "mixed": mixed,
            "total": total,
            "district": district
        }
    
    # Sort trucks by capacity
    available_trucks = sorted(trucks, key=lambda t: t.capacity, reverse=True)
    
    # First pass - assign specialized trucks to districts with matching waste types
    for district in districts_by_volume:
        if district.district_id in assignments:
            continue
            
        stats = district_stats[district.district_id]
        
        # Find the best specialized truck if the district has significant amounts of one waste type
        if stats["recyclable"] > stats["non_recyclable"] and stats["recyclable"] > stats["mixed"]:
            # Look for recyclable specialized truck
            for i, truck in enumerate(available_trucks):
                if truck.bin_type_specialty == "Recyclable":
                    route, distance = optimize_route_backtracking(truck, district.waste_bins, (0, 0))
                    if route:
                        truck.route = route
                        assignments[district.district_id] = {
                            "truck": truck,
                            "route": route,
                            "distance": distance
                        }
                        available_trucks.pop(i)
                        break
        
        elif stats["non_recyclable"] > stats["recyclable"] and stats["non_recyclable"] > stats["mixed"]:
            # Look for non-recyclable specialized truck
            for i, truck in enumerate(available_trucks):
                if truck.bin_type_specialty == "Non Recyclable":
                    route, distance = optimize_route_backtracking(truck, district.waste_bins, (0, 0))
                    if route:
                        truck.route = route
                        assignments[district.district_id] = {
                            "truck": truck,
                            "route": route,
                            "distance": distance
                        }
                        available_trucks.pop(i)
                        break
    
    # Second pass - assign remaining districts to remaining trucks
    for district in districts_by_volume:
        if district.district_id in assignments:
            continue
            
        if available_trucks:
            truck = available_trucks.pop(0)
            route, distance = optimize_route_backtracking(truck, district.waste_bins, (0, 0))
            
            if route:
                truck.route = route
                assignments[district.district_id] = {
                    "truck": truck,
                    "route": route,
                    "distance": distance
                }
    
    return assignments

## Data Loading Functions

In [ ]:
def load_districts_from_csv():
    """Load district data from CSV files"""
    districts = {}
    
    # Create default districts
    for i in range(1, 6):
        district_name = {
            1: "Downtown", 
            2: "Westside", 
            3: "Eastside", 
            4: "Northside", 
            5: "Southside"
        }
        districts[i] = District(i, district_name.get(i, f"District {i}"))
        
    # Default adjacency
    if len(districts) >= 5:
        districts[1].add_adjacent_district(districts[2])  # Downtown - Westside
        districts[1].add_adjacent_district(districts[3])  # Downtown - Eastside
        districts[1].add_adjacent_district(districts[4])  # Downtown - Northside
        districts[1].add_adjacent_district(districts[5])  # Downtown - Southside
        districts[2].add_adjacent_district(districts[4])  # Westside - Northside
        districts[3].add_adjacent_district(districts[4])  # Eastside - Northside
        districts[3].add_adjacent_district(districts[5])  # Eastside - Southside
            
    return list(districts.values())

def generate_synthetic_waste_bins(count=30):
    """Generate synthetic waste bins if real data not available"""
    waste_bins = []
    for i in range(1, count+1):
        x = random.uniform(0, 50)
        y = random.uniform(0, 50)
        
        capacity = random.choice([100, 200, 300])
        fill_level = random.uniform(20, 95)
        
        waste_bin = WasteBin(
            bin_id=f"bin-{i}",
            location=(x, y),
            fill_level=fill_level,
            capacity=capacity,
            bin_type=random.choice(["Recyclable", "Non Recyclable", "Mixed"]),
            container_type=random.choice(["Cubic", "Rectangular", "Silvertop-a"])
        )
        waste_bins.append(waste_bin)
    return waste_bins

def assign_bins_to_districts(districts, waste_bins):
    """Assign waste bins to districts based on proximity"""
    # Generate random centroids
    district_locations = {}
    for district in districts:
        district_locations[district.district_id] = (
            random.uniform(10, 40),
            random.uniform(10, 40)
        )
    
    # Assign bins to nearest district
    for bin in waste_bins:
        nearest_district = min(
            districts,
            key=lambda d: ((bin.location[0] - district_locations.get(d.district_id, (25, 25))[0])**2 + 
                          (bin.location[1] - district_locations.get(d.district_id, (25, 25))[1])**2)
        )
        nearest_district.add_waste_bin(bin)
    
    return districts

def create_trucks():
    """Create waste collection trucks with different capacities and specializations"""
    trucks = [
        Truck(1, 1200, "Recyclable"),     # Specialized in recyclables, large capacity
        Truck(2, 1000, "Non Recyclable"),  # Specialized in non-recyclables, medium capacity
        Truck(3, 1500),                   # General purpose, largest capacity
        Truck(4, 800, "Recyclable"),      # Another recyclable truck, smaller capacity
        Truck(5, 900)                     # General purpose, medium capacity
    ]
    return trucks

## Visualization Functions

In [ ]:
def visualize_routes(assignments):
    """Visualize truck routes on a map"""
    plt.figure(figsize=(12, 10))
    colors = ['red', 'green', 'blue', 'purple', 'orange', 'teal', 'brown']
    markers = {'Recyclable': 'o', 'Non Recyclable': 's', 'Mixed': '^'}
    
    # Plot each district's routes with a different color
    for i, (district_id, assignment) in enumerate(assignments.items()):
        route = assignment['route']
        color = colors[i % len(colors)]
        
        # Plot waste bins
        for bin in route:
            marker = markers.get(bin.bin_type, 'o')
            plt.scatter(bin.location[0], bin.location[1], color=color, 
                       marker=marker, s=100, alpha=0.7)
        
        # Plot route lines
        for j in range(len(route) - 1):
            plt.plot([route[j].location[0], route[j+1].location[0]],
                     [route[j].location[1], route[j+1].location[1]],
                     color=color, linewidth=2, alpha=0.5)
            
        # Mark bin IDs
        for bin in route:
            plt.annotate(bin.bin_id, bin.location, fontsize=8)
    
    # Add a legend for waste bin types
    for bin_type, marker in markers.items():
        plt.scatter([], [], marker=marker, color='black', 
                   label=f"{bin_type} waste")
    
    plt.title("Waste Collection Routes by District")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.legend()
    plt.grid(True)
    plt.show()

## Main Workflow

In [ ]:
# Set random seed for reproducibility
random.seed(42)

print("Smart Waste Management System")
print("-----------------------------")

In [ ]:
# Load data
districts = load_districts_from_csv()
waste_bins = generate_synthetic_waste_bins(100)
print(f"Generated {len(waste_bins)} waste bins")

In [ ]:
# Assign bins to districts
districts = assign_bins_to_districts(districts, waste_bins)

# Print district bin allocation
for district in districts:
    print(f"District {district.district_id} ({district.name}): {len(district.waste_bins)} bins")

In [ ]:
# Apply map coloring algorithm to segment districts
print("Applying map coloring algorithm to segment districts...")
colored_districts = map_coloring(districts)
visualize_districts(colored_districts)

In [ ]:
# Print district information
print("District Information:")
for district in colored_districts:
    print(f"District {district.district_id} ({district.name}): Color = {district.color}, " 
          f"Waste Bins = {len(district.waste_bins)}")
    
    # Calculate total waste volume per district
    total_waste = sum(bin.current_level for bin in district.waste_bins)
    emptying_needed = sum(1 for bin in district.waste_bins if bin.emptying_needed)
    print(f"  Total waste: {total_waste:.1f} units")
    print(f"  Bins needing emptying: {emptying_needed}/{len(district.waste_bins)}")
    print(f"  Waste types: Recyclable={sum(1 for b in district.waste_bins if b.bin_type=='Recyclable')}, " +
          f"Non-Recyclable={sum(1 for b in district.waste_bins if b.bin_type=='Non Recyclable')}, " +
          f"Mixed={sum(1 for b in district.waste_bins if b.bin_type=='Mixed')}")

In [ ]:
# Create trucks
trucks = create_trucks()
print(f"Created {len(trucks)} waste collection trucks")
for truck in trucks:
    specialty = truck.bin_type_specialty if truck.bin_type_specialty else "General"
    print(f"Truck {truck.truck_id}: Capacity = {truck.capacity}, Specialty = {specialty}")

In [ ]:
# Optimize routes and assign trucks
print("Optimizing routes using backtracking...")
assignments = assign_trucks_to_districts(trucks, colored_districts)

# Print assignment results
print("\nTruck Assignments:")
for district_id, assignment in assignments.items():
    district = next(d for d in districts if d.district_id == district_id)
    route = assignment['route']
    distance = assignment['distance']
    truck = assignment['truck']
    
    print(f"\nDistrict {district_id} ({district.name}) - Truck {truck.truck_id}")
    if truck.bin_type_specialty:
        print(f"  Truck specialty: {truck.bin_type_specialty}")
    print(f"  Route distance: {distance:.2f} units")
    print(f"  Waste bins to collect: {len(route)}")
    print(f"  Fill percentage of bins: {sum(bin.current_level for bin in route):.1f} / {truck.capacity} units")

In [ ]:
# Visualize routes
visualize_routes(assignments)

## Conclusion

This notebook demonstrates a smart waste management system that:

1. Segments cities into districts using map coloring algorithms
2. Optimizes collection routes using backtracking algorithms
3. Assigns appropriate trucks to districts based on waste types and volumes

The system can be extended with real-world data and additional optimization techniques.